In [1]:
# Import necessary libraries
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.noise import AlphaDropout
from keras.layers import Layer
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split

# Import custom modules
from network import *
from data import *

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Create the base network
network = dict(n_dense=5, dense_units=16, activation='selu', dropout=AlphaDropout, dropout_rate=0.1,
               kernel_initializer='lecun_normal', optimizer='sgd', num_classes=2)

shared_model = create_base_network(**network)

In [3]:
# Create the siamese network
model = create_siamese_network(shared_model)
print("Siamese network model summary")
model.summary()

Siamese network model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 978)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 978)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 16)           16752       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
man_dist_1 (ManDist)            (None, 1)            0           sequential_1[1

In [4]:
import pickle
dbfile = open('../../Data/X_train', 'rb')      
X_train = pickle.load(dbfile)
dbfile = open('../../Data/y_train', 'rb')      
y_train = pickle.load(dbfile)
dbfile = open('../../Data/X_test', 'rb')      
X_test = pickle.load(dbfile)
dbfile = open('../../Data/y_test', 'rb')      
y_test = pickle.load(dbfile)

In [5]:
model.fit([X_train[0],X_train[1]],y_train, verbose=0,epochs=30)

# Sandbox

In [6]:
X_train[0][1][0:10],X_train[1][1][0:10]

(array([-1.0255878 , -0.5268659 ,  0.93751162, -0.21547468,  0.56213748,
         0.19349912,  0.4320128 ,  0.19859782,  0.49415439,  1.27129149]),
 array([10.        ,  7.40159988, -2.45020008,  5.91870022,  7.57660007,
        -6.60559988, -3.51869988,  5.83519983, -6.07600021, -1.39619994]))

In [7]:
y_train[1]

0.0

In [8]:
pred = model.predict([X_train[0],X_train[1]])

In [9]:
pred.shape=(pred.shape[0],)

In [10]:
pred

array([0.05047047, 0.6860138 , 0.24235845, ..., 0.22565413, 0.6375929 ,
       0.06387591], dtype=float32)

In [11]:
min(pred)

0.002571702

In [12]:
max(pred)

35.487686

In [13]:
a = [-2.9628181, -2.9369824, -2.8793402, -2.917018 , -2.8999379,
         -2.934715 , -2.854669 , -2.8483472, -2.8471825, -2.8114026,
         -2.8795078, -2.9513776, -2.8242412, -2.8139703, -2.838598 ,
         -2.837599 ]
sum([i*i for i in a])

132.50516110874779

In [14]:
from keras.models import Model

for i in range(0,len(shared_model.layers)):
    intermediate_layer_model = Model(inputs=shared_model.get_input_at(0),
                                     outputs=[shared_model.layers[i].get_output_at(0)])

    intermediate_output = intermediate_layer_model.predict(X_train[0][11:15])
    print(i,"\n",intermediate_output[0],'\n',intermediate_output[1],'\n\n')

0 
 [-2.1254392 -0.6733637  4.181968   6.2469645  0.8739737 -1.4826963
 -3.3941326 -3.1367803 -1.9727012 -5.8120065 -5.3679767 -4.285265
 -5.535778   8.912957  -2.5861917 -7.324015 ] 
 [ -4.698345   -3.9079955  11.521126    4.481924   -7.057942   -8.330911
  -6.4726744  -3.7348802  -7.4908724  -9.794313   -6.9883494  -3.526871
 -17.82275    12.503428   -6.2925625  -9.9090805] 


1 
 [-1.5482165 -0.8614859  4.393998   6.563692   0.9182851 -1.3589673
 -1.6990806 -1.6817584 -1.5135819 -1.75284   -1.7499002 -1.7338904
 -1.7511669  9.364853  -1.6257035 -1.7569399] 
 [-1.7420825 -1.7227955 12.105259   4.709162  -1.7565864 -1.7576756
 -1.755383  -1.716123  -1.7571181 -1.7580013 -1.7564775 -1.706417
 -1.7580993 13.137365  -1.7548469 -1.7580119] 


2 
 [-1.5482165 -0.8614859  4.393998   6.563692   0.9182851 -1.3589673
 -1.6990806 -1.6817584 -1.5135819 -1.75284   -1.7499002 -1.7338904
 -1.7511669  9.364853  -1.6257035 -1.7569399] 
 [-1.7420825 -1.7227955 12.105259   4.709162  -1.7565864 -1.75767

In [17]:
shared_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                15664     
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
alpha_dropout_1 (AlphaDropou (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
alpha_dropout_2 (AlphaDropou (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
__________